In [1]:
import os
import time
import random
import tokenize
import multiprocessing
from tqdm import tqdm
from parser import remove_comments_and_docstrings

In [2]:
project_list = []
for dir1 in os.listdir('../py150_files/data/'):
    assert(os.path.isdir('../py150_files/data/' + dir1))
    for dir2 in os.listdir('../py150_files/data/' + dir1):
        assert(os.path.isdir('../py150_files/data/' + dir1 + '/' + dir2))
        project_list.append(dir1 + '/' + dir2)

In [3]:
def get_list(data_list, filename):
    file = open('../py150_files/' + filename + '.txt')
    context = file.read()
    file.close()
    file_list = context.strip().split('\n')
    
    for x in file_list:
        project = x[5 : x.find('/', x.find('/', 5) + 1)]
        assert(os.path.isdir('../py150_files/data/' + project))
        assert(os.path.isfile('../py150_files/' + x))
        if (os.path.isdir('../py150_files/structure/' + project)):
            if (os.path.isfile('../py150_files/structure/' + project + '/node.txt')):
                if (os.path.isfile('../py150_files/structure/' + project + '/edge.txt')):
                    data_list.append(x[5 :])

data_list = []
get_list(data_list, 'python50k_eval')
get_list(data_list, 'python100k_train')
data_list.sort()
print(len(data_list))

141349


In [4]:
def get_graph(path):
    nodes = {}
    edges = []
    root = -1
    
    file = open(path + '/node.txt')
    context = file.read()
    context = context.strip().split('\n')
    file.close()
    for x in context[1 :]:
        node_id = int(x[0 : x.find(',')])
        node_type = x[x.find(',') + 1 : x.find(',', x.find(',') + 1)]
        node_name = x[x.find(',', x.find(',') + 1) + 1 : x.rfind(',')]
        node_father = int(x[x.rfind(',') + 1 :])
        assert(node_id not in nodes)
        nodes[node_id] = (node_type, node_name, node_father)
    
    file = open(path + '/edge.txt')
    context = file.read()
    context = context.strip().split('\n')
    file.close()
    for x in context[1 :]:
        source = int(x[: x.find(',')])
        target = int(x[x.find(',') + 1 : x.find(',', x.find(',') + 1)])
        if ((source in nodes) and (target in nodes)):
            edges.append((source, target))
    for node_id in nodes:
        node_father = nodes[node_id][2]
        if (node_father in nodes):
            edges.append((node_father, node_id))
        else:
            assert(node_father == -1)

    E = {}
    for x in nodes:
        E[x] = []
    for (u, v) in edges:
        E[u].append(v)
        E[v].append(u)
    return nodes, edges, E

def find_root(nodes, root_name):
    root = -1
    for x in nodes:
        if (nodes[x][1][-len(root_name) :] == root_name):
            root = x
    return root

def cut_graph(nodes, edges, root, E):
    visited = set()
    visited.add(root)
    Q1 = [root]
    remain_nodes = [root]
    while (len(remain_nodes) < 200):
        Q2 = []
        for u in Q1:
            for v in E[u]:
                if (v not in visited):
                    visited.add(v)
                    Q2.append(v)
        if (len(Q2) == 0):
            break
        Q1 = Q2
        random.shuffle(Q1)
        for u in Q1:
            assert(u not in remain_nodes)
            if (u not in remain_nodes):
                node_type = nodes[u][0]
                node_father = nodes[u][2]
                if (node_father in nodes):
                    father_type = nodes[node_father][0]
                else:
                    father_type = 'File'
                if ((node_type != 'Variable') and (father_type in ['Package', 'File'])):
                    remain_nodes.append(u)
    remain_nodes = remain_nodes[: 200]
    
    new_nodes = []
    new_edges = []
    idmap = {}
    for i, x in enumerate(remain_nodes):
        new_nodes.append(nodes[x][1])
        idmap[x] = i
    for (u, v) in edges:
        if ((u in idmap) and (v in idmap)):
            new_edges.append((idmap[u], idmap[v]))
    return new_nodes, new_edges

def get_tokens(path, nodes, do_remove, idx):
    input_file = open(path)
    code = input_file.read()
    input_file.close()
    if (do_remove):
        code = remove_comments_and_docstrings(code, 'python')
    output_file = open('tmp' + str(idx) + '.py', 'w')
    print(code, file = output_file)
    output_file.close()
    
    tokens = []
    cross_edges = []
    f = open('tmp' + str(idx) + '.py', 'rb')
    tokenGenerator = tokenize.tokenize(f.readline)
    for token in tokenGenerator:
        if (token.type in [0, 59, 60, 62]): # COMMENT
            pass
        elif (token.type in [4, 61]): # NEWLINE
            pass
        elif (token.type == 5): # INDENT
            tokens.append('{')
            pass
        elif (token.type == 6): # DEDENT
            tokens.append('}')
            pass
        elif (token.type in [1, 2, 3, 54]): # NAME NUMBER STRING OP
            for i in range(len(nodes)):
                if (token.string == nodes[i]):
                    cross_edges.append((len(tokens), i))
            tokens.append(token.string)
        else:
            assert(False)
    f.close()
    return tokens, cross_edges

In [5]:
# one-thread

# output_file = open('../py150_files/washed_python150k.txt', 'w')
# last_project = ''
# for x in tqdm(data_list, total = len(data_list)):
#     project = x[: x.find('/', x.find('/') + 1)]
#     if (project == last_project):
#         nodes, edges, E = last_nodes, last_edges, last_E
#     else:
#         last_project = project
#         nodes, edges, E = get_graph('../py150_files/structure/' + project)
#         last_nodes, last_edges, last_E = nodes, edges, E
#     root = find_root(nodes, x)
#     if (root == -1):
#         continue
#     nodes, edges = cut_graph(nodes, edges, root, E)
#     try:
#         tokens, cross_edges = get_tokens('../py150_files/data/' + x, nodes, True, 0)
#     except:
#         tokens, cross_edges = get_tokens('../py150_files/data/' + x, nodes, False, 0)
#     os.system('rm tmp.py')
#     if (len(tokens) != 0):
#         data = {}
#         data['path'] = x
#         data['tokens'] = tokens
#         data['nodes'] = nodes
#         data['edges'] = edges
#         data['cross_edges'] = cross_edges
#         print(data, file = output_file)
# output_file.close()

In [6]:
# muti-thread

def work(path, idx):
    project = path[: path.find('/', path.find('/') + 1)]
    nodes, edges, E = get_graph('../py150_files/structure/' + project)
    root = find_root(nodes, path)
    if (root == -1):
        return 0
    nodes, edges = cut_graph(nodes, edges, root, E)
    try:
        tokens, cross_edges = get_tokens('../py150_files/data/' + path, nodes, True, idx)
    except:
        tokens, cross_edges = get_tokens('../py150_files/data/' + path, nodes, False, idx)
    os.system('rm tmp' + str(idx) + '.py')
    if (len(tokens) != 0):
        data = {}
        data['path'] = path
        data['tokens'] = tokens
        data['nodes'] = nodes
        data['edges'] = edges
        data['cross_edges'] = cross_edges
        return data
    else:
        return 0

start_time = time.time()
dataset = []
pool = multiprocessing.Pool(processes = 26)
for idx, path in enumerate(data_list):
    dataset.append(pool.apply_async(work, (path, idx, )))
pool.close()
pool.join()
print('using time =', round(time.time() - start_time, 2))

for i in range(len(dataset)):
    dataset[i] = dataset[i].get()

output_file = open('../py150_files/washed_python150k.txt2', 'w')
for data in dataset:
    if (data != 0):
        print(data, file = output_file)
output_file.close()

using time = 598.32
